In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import shutil
import wget
import time
import os

In [2]:
path = f'{os.getcwd()}/PDBs'
url_download = 'http://huanglab.phys.hust.edu.cn/pepbdb/db/download/pepbdb-20200318.tgz'
url = 'http://huanglab.phys.hust.edu.cn/pepbdb/db'

In [3]:
data = {
    'Peptide ID': [],
    'Sequence': [],
    'Length': [],
    'Num. Atoms in Peptide': [],
    'Protein Chain ID': [],
    'Receptor Sequence': [],
    'Num. Atoms in Protein': [],
    'Num. Atoms Contacts Between Peptide and Protein': [],
    'Resolution (-1.00 means NMR structure)': [],
    'Molecular Type': []
}

In [4]:
# Download Peptide List from PepBDB
filename = url_download.split('/')[len(url_download.split('/'))-1]
wget.download(url_download)
os.system(f'tar -xzf {filename} -C {path}/PepBDB_releases')

# Delete unnecessary files
os.remove(f'{os.getcwd()}/{filename}')
shutil.rmtree(f'{path}/PepBDB_releases/pepbdb')

100% [....................] 691391502 / 691391502

In [5]:
with open(f'{path}/PepBDB_releases/peptidelist.txt') as file:
    lines = file.readlines()
    for line in lines:
        values = line.split('\n')[0].split(' ')
        pep_info = [x.strip() for x in values if x != '']
        data['Peptide ID'].append(f'{pep_info[0]}_{pep_info[1]}')
        data['Length'].append(pep_info[2])
        data['Num. Atoms in Peptide'].append(pep_info[3])
        data['Protein Chain ID'].append(pep_info[4])
        data['Num. Atoms in Protein'].append(pep_info[5])
        data['Num. Atoms Contacts Between Peptide and Protein'].append(pep_info[6])
        data['Resolution (-1.00 means NMR structure)'].append(pep_info[9])
        if len(pep_info) == 11:
            data['Molecular Type'].append(pep_info[10])
        else:
            data['Molecular Type'].append('Not Found')

In [6]:
driver = webdriver.Chrome()

In [7]:
for pep_id in tqdm(data['Peptide ID']):
    driver.get(f'{url}/{pep_id}')
    time.sleep(2)
    
    try:
        peptide_seq = driver.find_element(By.XPATH, '/html/body/div[2]/table[3]/tbody/tr/td')
        pep_seq = ''
        for seq in peptide_seq.text.split('\n'):
            if not '>' in seq:
                pep_seq += seq
        data['Sequence'].append(pep_seq)
        
    except:
        data['Sequence'].append('Not Found')
    
    try:
        receptor_seq = driver.find_element(By.XPATH, '/html/body/div[2]/table[4]/tbody/tr/td')
        rec_seq = ''
        for seq in receptor_seq.text.split('\n'):
            if not '>' in seq:
                rec_seq += seq
        data['Receptor Sequence'].append(rec_seq)
    except:
        data['Receptor Sequence'].append('Not Found')

100%|███| 13301/13301 [16:35:14<00:00,  4.49s/it]


In [8]:
driver.close()

In [9]:
df = pd.DataFrame(data)

In [10]:
#Add links to peptides in PepBDB
links = []
for pep_id in df['Peptide ID']:
    links.append(f'{url}/{pep_id}')
df['Link'] = links

In [11]:
dt = datetime.now()
df.to_csv(f'{path}/PepBDB_releases/PepBDB_{dt.strftime("%Y_%m_%d")}.csv', index=False)
df.to_csv(f'{path}/PepBDB.csv', index=False)

In [12]:
df

,Peptide ID,Sequence,Length,Num. Atoms in Peptide,Protein Chain ID,Receptor Sequence,Num. Atoms in Protein,Num. Atoms Contacts Between Peptide and Protein,Resolution (-1.00 means NMR structure),Molecular Type,Link
0,1a1r_C,GSVVIVGRIVLSGKPA,16,108,A,VEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQM...,1117,90,2.50,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/1a1r_C
1,1a1u_A,EYFTLQIRGRERFEKIREYNEALELKDAQ,29,257,C,EYFTLQIRGRERFEKIREYNEALELKDAQ,257,121,-1.00,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/1a1u_A
2,1a2c_L,TFGSGEADCGLRPLFEKKSLEDKTERELLESYIDGR,36,281,H,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,2039,167,2.10,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/1a2c_L
3,1a2x_B,EEKRNRAITARRQHLKSVMLQIAATELEKEE,31,256,A,DQQAEARSYLSEEMIAEFKAAFDMFDADGGGDISVKELGTVMRMLG...,1246,107,2.30,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/1a2x_B
4,1a38_P,WLDLE,5,47,A,MDKNELVQKAKLAEQAERYDDMAACMKSVTEQGAELSNEERNLLSV...,1730,33,3.35,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/1a38_P
...,...,...,...,...,...,...,...,...,...,...,...
13296,6v92_6,XXXXXXXXXXXXXXX,15,75,O,KLVDFLVNVQSILNAASVKCHVVDESFPAKFFEKNPDKIYESYCKF...,3145,20,20.00,prot-nuc,http://huanglab.phys.hust.edu.cn/pepbdb/db/6v92_6
13297,6vdb_H,APRFGGVMRPNR,12,91,A,CVMDDFRDPQRWKECAKQGKMPCYFDLIEENVYLTERMQCECTPLS...,1821,87,2.30,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/6vdb_H
13298,6vfs_G,XXXXXXX,7,36,C,KLPTPAEIVANLNDHVIGQEQAKKALAVSVYNHYKRLRHPKAGANV...,2536,18,3.30,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/6vfs_G
13299,6vfx_G,XXXXXXX,7,34,A,KLPTPAEIVANLNDHVIGQEQAKKALAVSVYNHYKRLRHPKAGANV...,2604,27,2.90,prot,http://huanglab.phys.hust.edu.cn/pepbdb/db/6vfx_G
